In [33]:
import pandas as pd

# Load data from CSV files
movies = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\CS50-AI\Search\degrees\small\movies.csv')
people = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\CS50-AI\Search\degrees\small\people.csv')
stars = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\CS50-AI\Search\degrees\small\stars.csv')

# Standardizing column names between dataframes to enable merging
people.columns = ['person_id', 'name', 'birth']
movies.columns = ['movie_id', 'title', 'year']

# Merging dataframes to create a single dataframe with actor names and movie titles
stars = stars.merge(people, 'left', 'person_id').merge(movies, 'left', 'movie_id')
stars = stars[['name', 'title']]
stars.columns = ['actor', 'movie']

In [2]:
class Node:
    """
    A Node class representing a movie and its cast.
    """
    def __init__(self, movie, cast, parent_movie, parent_actor):
        self.movie = movie            # The movie title
        self.cast = cast              # List of actors in the movie
        self.parent_movie = parent_movie  # Parent movie node in the search path
        self.parent_actor = parent_actor  # Actor from the parent movie node

In [22]:
class Network:
    """
    A Network class built around an actor to find the degrees of separation between them and other actors.
    """
    def __init__(self, actor, stars):
        self.actor = actor
        self.stars = stars
        # Initialize the search frontier with the starting actor's movies
        self.frontier = [Node(movie,
                              list(self.stars.groupby('movie').get_group(movie)['actor']), 
                              None,
                              None) 
                         for movie in stars.groupby('actor').get_group(actor)['movie']]

    def find_degrees(self, target_actor):
        """
        Finds the degrees of separation between the initial actor and the target actor.
        """
        self.explored = set()
        # Mark the initial actor's movies as explored
        for movie in self.frontier:
            self.explored.add(movie.movie)

        solution = []
        while not solution:
            try:
                # Explore the next movie in the frontier
                explored_movie = self.frontier.pop(0)
            except IndexError:
                raise Exception('No connection found between actors')

            # Add neighboring movies to the frontier
            neighbor_movies = []
            for actor in explored_movie.cast:
                for movie in self.stars.groupby('actor').get_group(actor)['movie']:
                    if movie not in self.explored:
                        self.explored.add(movie)
                        neighbor_movies.append(Node(movie,
                                                    list(self.stars.groupby('movie').get_group(movie)['actor']),
                                                    explored_movie,
                                                    actor))

            # Check if the target actor is in the current movie's cast
            for movie in neighbor_movies:
                for actor in movie.cast:
                    if actor == target_actor:
                        # Build the solution path
                        parent_movie = movie
                        parent_actor = actor
                        while parent_movie:
                            solution.append((parent_movie.movie, parent_actor))
                            parent_actor = parent_movie.parent_actor
                            parent_movie = parent_movie.parent_movie
                        solution.reverse()
                        # Print the solution
                        print(f'{self.actor} is {len(solution)} {"degree" if len(solution) == 1 else "degrees"} from {target_actor}.')
                        print(f'{self.actor} starred in {solution[0][0]} with {solution[0][1]}.')
                        for i in range(1, len(solution)):
                            print(f'{solution[i-1][1]} starred in {solution[i][0]} with {solution[i][1]}')
                        break
                self.frontier.append(movie)

In [32]:
# Usage example (assuming the actors' names and data are in the loaded datasets)
network = Network('Tom Hanks', stars)
network.find_degrees('Dustin Hoffman')

Tom Hanks is 3 degrees from Dustin Hoffman.
Tom Hanks starred in Apollo 13 with Kevin Bacon.
Kevin Bacon starred in A Few Good Men with Tom Cruise
Tom Cruise starred in Rain Man with Dustin Hoffman
